## 双目相机标定实验
学号：23103402  
姓名：丁悦林

棋盘格宽度25mm，内角点11*7

In [1]:
import os
import glob
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

In [2]:
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [13]:
class StereoCameraCalibrator:
    def __init__(self, left_img_dir, right_img_dir, corner_shape, size):
        self.left_img_dir = left_img_dir
        self.right_img_dir = right_img_dir
        self.corner_shape = corner_shape
        self.size = size
        
        # 左右相机内参、畸变系数、旋转和平移向量
        self.left_inner_matrix = None
        self.left_dis_coeff = None
        self.left_vector_r = None
        self.left_vector_t = None
        
        self.right_inner_matrix = None
        self.right_dis_coeff = None
        self.right_vector_r = None
        self.right_vector_t = None
        
        # 双目标定结果
        self.R = None
        self.T = None
        self.E = None # 本征矩阵
        self.F = None # 基础矩阵
        
        # 准备棋盘格世界坐标
        w,h = corner_shape
        # cp指corner points
        cp_int = np.zeros((w*h, 3), np.float32)
        cp_int[:,:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)
        self.cp_world = cp_int * size  # (w*h,3):每一行为(X,Y,Z)
        
        self.left_img_paths = sorted(glob.glob(os.path.join(left_img_dir, "*.png")))
        self.right_img_paths = sorted(glob.glob(os.path.join(right_img_dir, "*.png")))
        assert len(self.left_img_paths), "左相机图像路径加载失败"
        assert len(self.right_img_paths), "右相机图像路径加载失败"
        assert len(self.left_img_paths) == len(self.right_img_paths), "左右相机图像数量不同"
    
    def _find_corners(self):
        """
        在左右相机图像中检测角点
        """
        w,h = self.corner_shape
        
        points_world = []
        left_points_pixel = [] # 存储像素坐标
        right_points_pixel = []
        left_imgs_with_corners = []
        right_imgs_with_corners = []
        img_size = None
        
        print(f"开始处理{len(self.left_img_paths)}对图像")
        
        for idx, (left_path,right_path) in enumerate(zip(self.left_img_paths, self.right_img_paths)):
            left_img = cv2.imread(left_path)
            right_img = cv2.imread(right_path)
            left_gray = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
            right_gray = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)
            
            # TODO:这个img_size实际上指的是什么？为什么左右图像使用同一个img_size
            if img_size is None: # 这里img_size图像标定的时候会用到
                img_size = left_gray.shape[::-1]
            
            # 检测左图角点
            ret_left, cp_left = cv2.findChessboardCorners(left_gray, (w, h), None)
            # 检测右图角点
            ret_right, cp_right = cv2.findChessboardCorners(right_gray, (w, h), None)
            
            if ret_left and ret_right:
                # 亚像素角点精细化
                criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
                # 得到角点的像素坐标
                cp_left = cv2.cornerSubPix(left_gray, cp_left, (11, 11), (-1, -1), criteria)
                cp_right = cv2.cornerSubPix(right_gray, cp_right, (11, 11), (-1, -1), criteria)
                
                points_world.append(self.cp_world)
                left_points_pixel.append(cp_left)
                right_points_pixel.append(cp_right)
                
                # 准备可视化
                left_img_corners = left_img.copy()
                right_img_corners = right_img.copy()
                cv2.drawChessboardCorners(left_img_corners, (w, h), cp_left, ret_left)
                cv2.drawChessboardCorners(right_img_corners, (w, h), cp_right, ret_right)
                
                left_imgs_with_corners.append(cv2.cvtColor(left_img_corners, cv2.COLOR_BGR2RGB))
                right_imgs_with_corners.append(cv2.cvtColor(right_img_corners, cv2.COLOR_BGR2RGB))
            else:
                print(f"第{idx+1}对图像角点检测失败")
        
        return points_world, left_points_pixel, right_points_pixel, \
               left_imgs_with_corners, right_imgs_with_corners, img_size
    
    def _visualize_corners(self, left_imgs_with_corners, right_imgs_with_corners):
        """
        可视化左右相机的角点检测结果
        """
        n_images = len(left_imgs_with_corners)
        cols = min(5, n_images)
        rows = (n_images+cols-1) // cols
        
        fig_left = plt.figure(figsize=(5*cols, 4*rows))
        for idx,img in enumerate(left_imgs_with_corners):
            ax = fig_left.add_subplot(rows, cols, idx+1)
            ax.imshow(img)
            ax.set_title(f"左{idx+1}", fontsize=15)
            ax.axis('off')
        plt.tight_layout()
        plt.suptitle("左相机角点检测结果", fontsize=15, y=1.00)
        plt.show()
        
        fig_right = plt.figure(figsize=(5*cols, 4*rows))
        for idx,img in enumerate(right_imgs_with_corners):
            ax = fig_right.add_subplot(rows, cols, idx+1)
            ax.imshow(img)
            ax.set_title(f"右{idx+1}", fontsize=15)
            ax.axis('off')
        plt.tight_layout()
        plt.suptitle("右相机角点检测结果", fontsize=15, y=1.00)
        plt.show()
    
    def stereo_calibrate_camera(self,flags=cv2.CALIB_USE_INTRINSIC_GUESS,visualize=False):
        """
        执行双目相机标定
        默认flags使用两步优化策略:
        1. 先分别对左右相机进行单目标定，获得初始内参和畸变系数
        2. 使用初始值进行双目联合优化，获得最终的内参和相对位姿
        """
        
        # 检测角点
        points_world, left_points_pixel, right_points_pixel, \
        left_imgs_corners, right_imgs_corners, img_size = self._find_corners()
        if len(points_world) == 0:
            raise ValueError("没有检测到有效的角点")
        
        if visualize:
            self._visualize_corners(left_imgs_corners, right_imgs_corners)
        
        # 左右相机单目标定
        ret_left, left_mtx, left_dist, left_rvecs, left_tvecs = cv2.calibrateCamera(
            points_world, left_points_pixel, img_size, None, None
        )
        print(f"左相机重投影误差:{ret_left:.4f}像素")
        ret_right, right_mtx, right_dist, right_rvecs, right_tvecs = cv2.calibrateCamera(
            points_world, right_points_pixel, img_size, None, None
        )
        print(f"右相机重投影误差:{ret_right:.4f}像素")
        
        # 使用单目标定结果作为初值，双目联合标定
        criteria_stereo = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6)
        ret_stereo, left_mtx_final, left_dist_final, right_mtx_final, right_dist_final, \
        R, T, E, F = cv2.stereoCalibrate(
            points_world, left_points_pixel, right_points_pixel,
            left_mtx, left_dist, # 左右相机内参和畸变系数的初值
            right_mtx, right_dist,
            img_size,
            criteria=criteria_stereo,
            flags=flags
        )
        print(f"双目联合标定重投影误差:{ret_stereo:.4f}像素")
        
        # 保存标定结果
        self.left_inner_matrix = left_mtx_final
        self.left_dis_coeff = left_dist_final
        self.left_vector_r = left_rvecs
        self.left_vector_t = left_tvecs
        
        self.right_inner_matrix = right_mtx_final
        self.right_dis_coeff = right_dist_final
        self.right_vector_r = right_rvecs
        self.right_vector_t = right_tvecs
        
        self.R = R
        self.T = T
        self.E = E
        self.F = F

        return ret_stereo, left_mtx_final, left_dist_final, \
               right_mtx_final, right_dist_final, R, T, E, F

In [12]:
left_img_dir = "../data/imgs/leftcamera"
right_img_dir = "../data/imgs/rightcamera"
corner_shape = (11,7)
size = 0.025

In [15]:
stero_cammera_calibrator = StereoCameraCalibrator(left_img_dir,right_img_dir,corner_shape,size)
flags1 = cv2.CALIB_USE_INTRINSIC_GUESS # 使用输入的内参和畸变系数作为优化的初值
flags2 = cv2.CALIB_FIX_INTRINSIC # 完全固定输入的内参和畸变系数
flags3 = 0 # 忽略输入的内参和畸变系数，重新计算

In [ ]:
stero_cammera_calibrator.stereo_calibrate_camera(flags=flags1)
print()
stero_cammera_calibrator.stereo_calibrate_camera(flags=flags2)
print()
stero_cammera_calibrator.stereo_calibrate_camera(flags=flags3)